# Ceneo Scraper

1. Import bibliotek

In [6]:
import requests
from bs4 import BeautifulSoup

2. Strona produktu

In [7]:
produkt_id = "84514582"
url = f"https://www.ceneo.pl/{produkt_id}#tab=reviews"

3. Wysłanie żądania dostepu do zasobu

In [9]:
response = requests.get(url)
response.status_code


200

4. Pobranie  opinii z HTML strony

In [10]:
page = BeautifulSoup(response.text, "html.parser")
opinions = page.select("div.js_product-review")
opinion = page.select_one("div.js_product-review")
print(type(opinions))
print(type(opinion))
print(opinion)

<class 'bs4.element.ResultSet'>
<class 'bs4.element.Tag'>
<div class="user-post user-post__card js_product-review" data-entry-id="12535429">
<header class="user-post__header">
<div class="js_lazy user-post__avatar user-rank__avatar" data-bg="/Content/img/account/avatar/8.svg"></div>
</header>
<div class="user-post__body">
<div class="user-post__content">
<span class="user-post__author-name">
g...6</span>
<span class="user-post__score">
<span class="screen-reader-text">Ocena:</span>
<span class="score-container score-container--s js_score-container">
<span class="score-marker score-marker--s" style="width: 60.00%;"></span>
</span>
<span class="user-post__score-count">3/5</span>
<span class="user-post__published">
        Wystawiono
        <time datetime="2020-05-28 08:02:51">4 lata temu, </time>
<time datetime="2020-05-04 22:29:44">po miesiącu</time> użytkowania            </span>
</span>
<div class="user-post__text">Instalacja łatwa. Oprogramowanie HP Smart bezproblemowe. Zwrot 100zl 

5. Pobranie składowych pojedynczej opini

|składowa|zmienna|selektor|
|--------|-------|--------|
|identyfikator|opinion_id|["data-entry-id"]|
|autora|author|span.user-post__author-name|
|rekomendację|recomendation|span.user-post__author-recomendation > em|
|liczbę gwiazdek|rating|span.user-post__score-count|
|data wystawienia opinii|opinion_date|span.user-post__published > time:nth-child(1)["datetime"]|
|data zakupu produktu|purchase_date|span.user-post__published > time:nth-child(2)["datetime"]|
|ile osób uznało opinię za przydatną|likes|button.vote-yes > span|
|ile osób uznało opinię za nieprzydatną|dislikes|buton.vote-no > span|
|treść opinii|content|div.user-post__text|
|listę wad|cons|div.review-feature__title--negatives ~ review-feature__item|
|listę zalet|pros|div.review-feature__title--positives ~ review-feature__item|

In [20]:
author = opinion.select_one("span.user-post__author-name").get_text().strip()
# recomendation = opinion.select_one("span.user-post__author-recomendation > em").get_text().strip()
rating = opinion.select_one("span.user-post__score-count").get_text().strip()
content = opinion.select_one("div.user-post__text").get_text().strip()
pros = [p.get_text().strip() for p in opinion.select("div.review-feature__title--positives ~ review-feature__item")]
# pros = opinion.select_one("div.review-feature__title--positives ~ review-feature__item").get_text().strip()
cons = [c.get_text().strip() for c in opinion.select("div.review-feature__title--negatives ~ review-feature__item")]
opinion_date = opinion.select_one("span.user-post__published > time:nth-child(1)")['datetime'].strip()
purchase_date= opinion.select_one("span.user-post__published > time:nth-child(2)")['datetime'].strip()
likes = opinion.select_one("button.vote-yes > span").get_text().strip()
dislikes = opinion.select_one("button.vote-no > span").get_text().strip()
opinion_id = opinion["data-entry-id"].strip()
